<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Risk Allocation

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

## Risk Contributions

Topics of interest include:

* marginal volatilities
* risk contributions (absolute)
* risk contributions (relative)
* risk parity portfolios (equal risk contributions)
* other risk allocations (random risk contributions)
* risk parity portfolios with target risk

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon.

The data sets:

    http://hilpisch.com/indices_eikon_eod_data.csv
    http://hilpisch.com/aiif_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_eod_data.csv

## Imports and Data

In [ ]:
from mvp_portfolio import *
from risk_measures import *

In [ ]:
# url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
url = 'http://hilpisch.com/indices_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
indices = [
    '.FCHI',  # France CAC40
    '.FTSE',  # Britain FTSE100
    '.GDAXI', # Germany DAX30
    '.HSI',  # Hong Kong HSI
    '.NDX',  # USA NASDAQ100
    '.NSEI',  # India NIFTY50
    '.SPX',  # USA S&P500
    '.SSEC'  # China Shanghei Composite Index
]

In [ ]:
universe = raw.columns[:3]
universe

In [ ]:
data = raw[universe]

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
noa = len(universe)

## Equally Weighted Portfolio

In [ ]:
w = np.array(noa * [1 / noa])
w

In [ ]:
cov = rets.cov() * 252

In [ ]:
cov

In [ ]:
vol = portfolio_volatility(w, rets)
vol

In [ ]:
vol / noa  # "average" risk contribution

In [ ]:
np.dot(cov, w)

In [ ]:
mvols = np.dot(cov, w) / vol  # marginal volatilities
mvols

In [ ]:
rc = mvols * w  # risk contributions (%-points)
rc

In [ ]:
rc.sum()

In [ ]:
rc / rc.sum()  # relative risk contributions (%)

In [ ]:
sum(rc / rc.sum())  # checking for sum of relative risk contributions

In [ ]:
np.dot(mvols, w) # portfolio volatility

## Risk Parity Portfolios

In [ ]:
from scipy.optimize import minimize

In [ ]:
def rel_risk_contributions(weights, rets=rets):
    vol = portfolio_volatility(weights, rets)
    cov = rets.cov()
    mvols = np.dot(cov, weights) / vol
    rc = mvols * weights
    rc = rc / rc.sum()
    return rc

In [ ]:
rel_risk_contributions(w)

In [ ]:
def mse_risk_contributions(weights, target, rets=rets):
    rc = rel_risk_contributions(weights, rets)
    mse = ((rc - target) ** 2).mean()
    return mse * 100

In [ ]:
mse_risk_contributions(w, w)

In [ ]:
bnds = noa * [(0, 1),]

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
target = noa * [1 / noa,]  # risk parity as target
target

In [ ]:
opt = minimize(lambda w: mse_risk_contributions(w, target=target),
               w, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
rel_risk_contributions(opt['x'])

In [ ]:
portfolio_volatility(opt['x'], rets)

## Random Risk Contributions

In [ ]:
target = np.random.random(noa)

In [ ]:
target /= target.sum()

In [ ]:
target

In [ ]:
target.sum()

In [ ]:
opt = minimize(lambda w: mse_risk_contributions(w, target=target),
               w, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
rel_risk_contributions(opt['x'])

In [ ]:
rel_risk_contributions(opt['x']) - target

In [ ]:
portfolio_volatility(opt['x'], rets)

## Target Volatility

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights:
        portfolio_volatility(weights, rets) - 0.15}

In [ ]:
target = noa * [1 / noa,]

In [ ]:
opt = minimize(lambda w: mse_risk_contributions(w, target=target),
               w, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
opt['x'].sum()

In [ ]:
rel_risk_contributions(opt['x'])

In [ ]:
portfolio_volatility(opt['x'], rets)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>